#Importaciones Iniciales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

### **1. Importaciones Iniciales**

In [ ]:
#Importaciones Iniciales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import re
import datetime 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data/block_0.csv')
data.head()

,LCLid,tstp,energy(kWh/hh)
0,MAC000002,2012-10-12 00:30:00.0000000,0
1,MAC000002,2012-10-12 01:00:00.0000000,0
2,MAC000002,2012-10-12 01:30:00.0000000,0
3,MAC000002,2012-10-12 02:00:00.0000000,0
4,MAC000002,2012-10-12 02:30:00.0000000,0


### **2. Inspeccion Inicial de la Data**

In [ ]:
#Inspeccion Inicial de la Info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222670 entries, 0 to 1222669
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   LCLid           1222670 non-null  object
 1   tstp            1222670 non-null  object
 2   energy(kWh/hh)  1222670 non-null  object
dtypes: object(3)
memory usage: 28.0+ MB


In [ ]:
data.shape

(1222670, 3)

In [ ]:
#Este dataset contiene: 
# 0   LCLid          object  =>Id del medidor instalado en el bloque/Casa?                            Var Obj
# 1   day            object  =>Dia de la medicion                                                     Var Obj
# 2   energy_median  float64 =>Mediana de la energia consumida durante el dia registrado              Var Continua
# 3   energy_mean    float64 =>Media de la energia consumida durante el dia registrado                Var Continua
# 4   energy_max     float64 =>Max de la energia consumida durante el dia registrado                  Var Continua
# 5   energy_count   int64   =>Numero de registros tomados durante el dia (cada 30 min son 48 al dia) Var Discreta
# 6   energy_std     float64 =>Desviacion Std                                                         Var Continua
# 7   energy_sum     float64 =>Suma del consumo diario                                                Var Continua
# 8   energy_min     float64 =>Max de la energia consumida durante el dia registrado                  Var Continua

In [ ]:
data.columns

Index(['LCLid', 'tstp', 'energy(kWh/hh)'], dtype='object')

In [ ]:
data.describe()

,LCLid,tstp,energy(kWh/hh)
count,1222670,1222670,1222670
unique,50,39292,5022
top,MAC000246,2012-12-16 23:30:00.0000000,0.013
freq,39245,50,6238


In [ ]:
#Analizar los Nulos 
null_cols = data.isnull().sum()
null_cols

LCLid             0
tstp              0
energy(kWh/hh)    0
dtype: int64

In [ ]:
data_clean = data.copy()

In [ ]:
data_clean.head()

,LCLid,tstp,energy(kWh/hh)
0,MAC000002,2012-10-12 00:30:00.0000000,0
1,MAC000002,2012-10-12 01:00:00.0000000,0
2,MAC000002,2012-10-12 01:30:00.0000000,0
3,MAC000002,2012-10-12 02:00:00.0000000,0
4,MAC000002,2012-10-12 02:30:00.0000000,0


In [ ]:
#Identificar elementos de baja varianza 
low_variance = []

for col in data_clean._get_numeric_data():
  minimun = min(data_clean[col])
  ninety_perc = np.percentile(data_clean[col], 90)
  if ninety_perc == minimun:
    low_variance.append(col)


print(low_variance)

[]


In [ ]:
#En este caso, la c

In [ ]:
data_clean.describe()

,LCLid,tstp,energy(kWh/hh)
count,1222670,1222670,1222670
unique,50,39292,5022
top,MAC000246,2012-12-16 23:30:00.0000000,0.013
freq,39245,50,6238


In [ ]:
#Diagrama de Bigotes

for col in data_clean._get_numeric_data():
  print(col)
  print(data_clean[col].plot.box())
  plt.show()
  print('------' * 10)

In [ ]:
#El diagrama de bigotes de energy_sum nos muestra que consumo mayor a 50 puede ser un outlier. (el 90% de mis datos va a ser 40)

In [ ]:
#Elegir el rango adecuado de analisis (No medidores instalados vs año

In [ ]:
#Preparar Columna de Day para manipulacion. 
data_clean['Date'] = pd.to_datetime(data_clean['tstp']
                            # , format='%d/%m/%Y'
                            # , dayfirst=True
                            )

In [ ]:
data_clean.head(2)

,LCLid,tstp,energy(kWh/hh),Date
0,MAC000002,2012-10-12 00:30:00.0000000,0,2012-10-12 00:30:00
1,MAC000002,2012-10-12 01:00:00.0000000,0,2012-10-12 01:00:00


In [ ]:
#Agregar columnas de Day, Month, Year, Q para analisis posterior
data_clean['Days'] = data_clean['Date'].dt.day
data_clean['Month'] = data_clean['Date'].dt.month
data_clean['Year'] = data_clean['Date'].dt.year
data_clean['Q'] = data_clean['Date'].dt.to_period(freq='Q')
data_clean.head(2)

,LCLid,tstp,energy(kWh/hh),Date,Days,Month,Year,Q
0,MAC000002,2012-10-12 00:30:00.0000000,0,2012-10-12 00:30:00,12,10,2012,2012Q4
1,MAC000002,2012-10-12 01:00:00.0000000,0,2012-10-12 01:00:00,12,10,2012,2012Q4


In [ ]:
data_clean['W'] = data_clean['Date'].dt.to_period(freq='W')
data_clean['Day'] = data_clean['Date'].dt.to_period(freq='D')
data_clean['Hour'] = data_clean['Date'].dt.to_period(freq='H')
data_clean.head(2)

,LCLid,tstp,energy(kWh/hh),Date,Days,Month,Year,Q,W,Day,Hour
0,MAC000002,2012-10-12 00:30:00.0000000,0,2012-10-12 00:30:00,12,10,2012,2012Q4,2012-10-08/2012-10-14,2012-10-12,2012-10-12 00:00
1,MAC000002,2012-10-12 01:00:00.0000000,0,2012-10-12 01:00:00,12,10,2012,2012Q4,2012-10-08/2012-10-14,2012-10-12,2012-10-12 01:00


In [ ]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1222670 entries, 0 to 1222669
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   LCLid           1222670 non-null  object        
 1   tstp            1222670 non-null  object        
 2   energy(kWh/hh)  1222670 non-null  object        
 3   Date            1222670 non-null  datetime64[ns]
 4   Days            1222670 non-null  int64         
 5   Month           1222670 non-null  int64         
 6   Year            1222670 non-null  int64         
 7   Q               1222670 non-null  period[Q-DEC] 
 8   W               1222670 non-null  period[W-SUN] 
 9   Day             1222670 non-null  period[D]     
 10  Hour            1222670 non-null  period[H]     
dtypes: datetime64[ns](1), int64(3), object(3), period[D](1), period[H](1), period[Q-DEC](1), period[W-SUN](1)
memory usage: 102.6+ MB


In [ ]:
#Se incorpora dataset de Tarifas preparado por CM. 
data_holydays = pd.read_csv('/content/drive/MyDrive/Data/uk_bank_holidays.csv')

In [ ]:
#Consumo Energetico Vista Cliente
#Consumo por dia, mes, ano, dia feriado, 
#1. Seleccionando Muestra ano 2013
muestra_anual = data_clean[data_clean.loc[:, 'Year'] == 2013]
muestra_anual_esp = muestra_anual[muestra_anual.loc[:, 'LCLid'] == 'MAC000002']
muestra_anual_esp


,LCLid,tstp,energy(kWh/hh),Date,Days,Month,Year,Q,W,Day,Hour
3837,MAC000002,2013-01-01 00:00:00.0000000,0.219,2013-01-01 00:00:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 00:00
3838,MAC000002,2013-01-01 00:30:00.0000000,0.241,2013-01-01 00:30:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 00:00
3839,MAC000002,2013-01-01 01:00:00.0000000,0.191,2013-01-01 01:00:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 01:00
3840,MAC000002,2013-01-01 01:30:00.0000000,0.235,2013-01-01 01:30:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 01:00
3841,MAC000002,2013-01-01 02:00:00.0000000,0.182,2013-01-01 02:00:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 02:00
...,...,...,...,...,...,...,...,...,...,...,...
21352,MAC000002,2013-12-31 21:30:00.0000000,0.451,2013-12-31 21:30:00,31,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-31,2013-12-31 21:00
21353,MAC000002,2013-12-31 22:00:00.0000000,0.456,2013-12-31 22:00:00,31,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-31,2013-12-31 22:00
21354,MAC000002,2013-12-31 22:30:00.0000000,0.402,2013-12-31 22:30:00,31,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-31,2013-12-31 22:00
21355,MAC000002,2013-12-31 23:00:00.0000000,0.491,2013-12-31 23:00:00,31,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-31,2013-12-31 23:00


In [ ]:
#Tratamiento de Consumo de Energia para pasarlo a numero
number_to_clean = muestra_anual_esp['energy(kWh/hh)'].values
number_to_clean_list = list(number_to_clean)
text_cleaned = [(i.strip()) for i in number_to_clean_list ]
number_cleaned = [(float(i)) for i in text_cleaned ]
muestra_anual_esp['Consumo(kWh/hh)'] = number_cleaned


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
muestra_anual_esp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 3837 to 21356
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   LCLid            17520 non-null  object        
 1   tstp             17520 non-null  object        
 2   energy(kWh/hh)   17520 non-null  object        
 3   Date             17520 non-null  datetime64[ns]
 4   Days             17520 non-null  int64         
 5   Month            17520 non-null  int64         
 6   Year             17520 non-null  int64         
 7   Q                17520 non-null  period[Q-DEC] 
 8   W                17520 non-null  period[W-SUN] 
 9   Day              17520 non-null  period[D]     
 10  Hour             17520 non-null  period[H]     
 11  Consumo(kWh/hh)  17520 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(3), period[D](1), period[H](1), period[Q-DEC](1), period[W-SUN](1)
memory usage: 1.7+ MB


In [ ]:
muestra_anual_esp.head()

,LCLid,tstp,energy(kWh/hh),Date,Days,Month,Year,Q,W,Day,Hour,Consumo(kWh/hh)
3837,MAC000002,2013-01-01 00:00:00.0000000,0.219,2013-01-01 00:00:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 00:00,0.219
3838,MAC000002,2013-01-01 00:30:00.0000000,0.241,2013-01-01 00:30:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 00:00,0.241
3839,MAC000002,2013-01-01 01:00:00.0000000,0.191,2013-01-01 01:00:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 01:00,0.191
3840,MAC000002,2013-01-01 01:30:00.0000000,0.235,2013-01-01 01:30:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 01:00,0.235
3841,MAC000002,2013-01-01 02:00:00.0000000,0.182,2013-01-01 02:00:00,1,1,2013,2013Q1,2012-12-31/2013-01-06,2013-01-01,2013-01-01 02:00,0.182


In [ ]:
#2. Consumo por dia y stats (Solo muestra del ano 2013)
consumo_clean_dia_cliente = muestra_anual_esp.groupby('Day').agg({'Consumo(kWh/hh)': 'sum'})
consumo_clean_dia_cliente


,Consumo(kWh/hh)
Day,
2013-01-01,10.800
2013-01-02,13.300
2013-01-03,10.074
2013-01-04,9.857
2013-01-05,11.171
...,...
2013-12-27,9.122
2013-12-28,13.658
2013-12-29,14.982


In [ ]:
consumo_clean_dia_cliente.describe()

,Consumo(kWh/hh)
count,365.000000
mean,11.546981
std,4.579990
min,4.548000
25%,8.969000
50%,10.727000
75%,12.918000
max,39.284000


In [ ]:
#3. Consumo por mes y stats (Solo muestra del ano 2013)
consumo_clean_mes_cliente = muestra_anual_esp.groupby('Month').agg({'Consumo(kWh/hh)': 'sum'})
consumo_clean_mes_cliente


,Consumo(kWh/hh)
Month,
1,359.872000
2,381.622000
3,479.684000
4,362.588001
5,303.894000
6,299.931000
7,271.151000
8,269.891000
9,321.512000


In [ ]:
consumo_clean_dia_cliente.describe()

,Consumo(kWh/hh)
count,365.000000
mean,11.546981
std,4.579990
min,4.548000
25%,8.969000
50%,10.727000
75%,12.918000
max,39.284000


In [ ]:
#5. Consumo por Q y stats (Solo muestra del ano 2013)
consumo_clean_quarter_cliente = muestra_anual_esp.groupby('Q').agg({'Consumo(kWh/hh)': 'sum'})
consumo_clean_quarter_cliente


,Consumo(kWh/hh)
Q,
2013Q1,1221.178000
2013Q2,966.413001
2013Q3,862.554000
2013Q4,1164.503000


In [ ]:
# Consumo dia normal segun muestra 

In [ ]:
muestra_dia_normal = muestra_anual_esp[muestra_anual_esp.loc[:, 'Day'] == '2013-12-30']
muestra_dia_normal

#El consumo en este dia fue 15.082

,LCLid,tstp,energy(kWh/hh),Date,Days,Month,Year,Q,W,Day,Hour,Consumo(kWh/hh)
21261,MAC000002,2013-12-30 00:00:00.0000000,0.494,2013-12-30 00:00:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 00:00,0.494
21262,MAC000002,2013-12-30 00:30:00.0000000,0.452,2013-12-30 00:30:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 00:00,0.452
21263,MAC000002,2013-12-30 01:00:00.0000000,0.47,2013-12-30 01:00:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 01:00,0.470
21264,MAC000002,2013-12-30 01:30:00.0000000,0.417,2013-12-30 01:30:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 01:00,0.417
21265,MAC000002,2013-12-30 02:00:00.0000000,0.16,2013-12-30 02:00:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 02:00,0.160
21266,MAC000002,2013-12-30 02:30:00.0000000,0.105,2013-12-30 02:30:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 02:00,0.105
21267,MAC000002,2013-12-30 03:00:00.0000000,0.101,2013-12-30 03:00:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 03:00,0.101
21268,MAC000002,2013-12-30 03:30:00.0000000,0.116,2013-12-30 03:30:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 03:00,0.116
21269,MAC000002,2013-12-30 04:00:00.0000000,0.085,2013-12-30 04:00:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 04:00,0.085
21270,MAC000002,2013-12-30 04:30:00.0000000,0.124,2013-12-30 04:30:00,30,12,2013,2013Q4,2013-12-30/2014-01-05,2013-12-30,2013-12-30 04:00,0.124


In [ ]:
#Analizando el dia del consumo maximo. 
muestra_dias_maximo = consumo_clean_dia_cliente[consumo_clean_dia_cliente.loc[:, 'Consumo(kWh/hh)'] > 20]
muestra_dias_maximo


,Consumo(kWh/hh)
Day,
2013-02-21,20.696
2013-02-22,24.018
2013-02-28,32.628
2013-03-01,39.284
2013-03-02,26.292
2013-03-03,23.580
2013-03-04,25.447
2013-03-05,32.856
2013-03-07,27.718


In [ ]:
muestra_dia_max_hora = muestra_anual_esp[muestra_anual_esp.loc[:, 'Day'] == '2013-03-01']
muestra_dia_max_hora

,LCLid,tstp,energy(kWh/hh),Date,Days,Month,Year,Q,W,Day,Hour,Consumo(kWh/hh)
6669,MAC000002,2013-03-01 00:00:00.0000000,0.483,2013-03-01 00:00:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 00:00,0.483
6670,MAC000002,2013-03-01 00:30:00.0000000,0.339,2013-03-01 00:30:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 00:00,0.339
6671,MAC000002,2013-03-01 01:00:00.0000000,0.299,2013-03-01 01:00:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 01:00,0.299
6672,MAC000002,2013-03-01 01:30:00.0000000,0.263,2013-03-01 01:30:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 01:00,0.263
6673,MAC000002,2013-03-01 02:00:00.0000000,0.225,2013-03-01 02:00:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 02:00,0.225
6674,MAC000002,2013-03-01 02:30:00.0000000,0.245,2013-03-01 02:30:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 02:00,0.245
6675,MAC000002,2013-03-01 03:00:00.0000000,0.246,2013-03-01 03:00:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 03:00,0.246
6676,MAC000002,2013-03-01 03:30:00.0000000,0.216,2013-03-01 03:30:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 03:00,0.216
6677,MAC000002,2013-03-01 04:00:00.0000000,0.185,2013-03-01 04:00:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 04:00,0.185
6678,MAC000002,2013-03-01 04:30:00.0000000,0.085,2013-03-01 04:30:00,1,3,2013,2013Q1,2013-02-25/2013-03-03,2013-03-01,2013-03-01 04:00,0.085


In [ ]:
#PasoFinal Exportaciones a CSV
#df.to_csv('/content/drive/MyDrive/Data/DataFinal/name_csv_file.csv')

#MuestraConsumoClienteTodoelAno 2013
muestra_anual_esp.to_csv('/content/drive/MyDrive/Data/DataFinal/consumo_cliente_ano.csv')

#ConsumoClienteporDia
consumo_clean_dia_cliente.to_csv('/content/drive/MyDrive/Data/DataFinal/consumo_cliente_dia.csv')

#ConsumoClienteMes
consumo_clean_mes_cliente.to_csv('/content/drive/MyDrive/Data/DataFinal/consumo_cliente_mes.csv')

#ConsumoTrimestre
consumo_clean_quarter_cliente.to_csv('/content/drive/MyDrive/Data/DataFinal/consumo_cliente_trimestre.csv')

#MuestraConsumoparaelDia (2013-12-30)  #El consumo en este dia fue 15.082
muestra_dia_normal.to_csv('/content/drive/MyDrive/Data/DataFinal/consumo_cliente_dia_normal.csv')

#MuestraConsumoparaelDiaMamximo (2013-03-01)  #El consumo en este dia fue 39.284
muestra_dia_max_hora.to_csv('/content/drive/MyDrive/Data/DataFinal/consumo_cliente_dia_max.csv')

#Muestradevalores mayores consumos mayores a 20
muestra_dias_maximo.to_csv('/content/drive/MyDrive/Data/DataFinal/consumo_cliente_mayores_consumos.csv')